Okay, so the better way to do this would be to actually load this data into postgres and then query it in the database, but for the sake of actually getting the data out we'll do it the slow way and load it into jupyter notebook using pandas.

In [55]:
import pandas as pd
import pickle

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

import re
import PyPDF2
import pickle

from functools import reduce

In [3]:
brfss_2014 = pd.read_csv("../data/brfss/brfss2014.csv", encoding = "cp1252", nrows=100)

In [5]:
brfss_2014.head()

,Unnamed: 0,x.state,fmonth,idate,imonth,iday,iyear,dispcode,seqno,x.psu,...,x.fobtfs,x.crcrec,x.aidtst3,x.impeduc,x.impmrtl,x.imphome,rcsbrac1,rcsrace1,rchisla1,rcsbirth
0,1,1,1,1172014,1,17,2014,1100,2014000001,2014000001,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN
1,2,1,1,1072014,1,7,2014,1100,2014000002,2014000002,...,2.0,2.0,2.0,4,1,1,NaN,NaN,NaN,NaN
2,3,1,1,1092014,1,9,2014,1100,2014000003,2014000003,...,2.0,2.0,2.0,6,1,1,NaN,NaN,NaN,NaN
3,4,1,1,1072014,1,7,2014,1100,2014000004,2014000004,...,2.0,1.0,2.0,6,3,1,NaN,NaN,NaN,NaN
4,5,1,1,1162014,1,16,2014,1100,2014000005,2014000005,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN


In [30]:
brfss_2014.rename(columns={'Unnamed: 0': 'record_id'}, inplace=True)

remove_periods_dict = dict([(col, col.replace(".", "")) for col in brfss_2014.columns])

brfss_2014.rename(columns=remove_periods_dict, inplace=True)

In [33]:
brfss_2014_columns = [str(col).lower() + " TEXT" for col in brfss_2014.columns]

brfss_2014_columns_str = ", ".join(brfss_2014_columns)
brfss_2014_create_table_query = "CREATE TABLE brfss_2014 (" + brfss_2014_columns_str + ")"

brfss_2014_load_table = "COPY brfss_2014 FROM '/home/ubuntu/brfss/brfss2014.csv' CSV HEADER ENCODING 'LATIN1'"

In [34]:
brfss_2014_create_table_query

'CREATE TABLE brfss_2014 (record_id TEXT, xstate TEXT, fmonth TEXT, idate TEXT, imonth TEXT, iday TEXT, iyear TEXT, dispcode TEXT, seqno TEXT, xpsu TEXT, ctelenum TEXT, pvtresd1 TEXT, colghous TEXT, stateres TEXT, ladult TEXT, numadult TEXT, nummen TEXT, numwomen TEXT, genhlth TEXT, physhlth TEXT, menthlth TEXT, poorhlth TEXT, hlthpln1 TEXT, persdoc2 TEXT, medcost TEXT, checkup1 TEXT, exerany2 TEXT, sleptim1 TEXT, cvdinfr4 TEXT, cvdcrhd4 TEXT, cvdstrk3 TEXT, asthma3 TEXT, asthnow TEXT, chcscncr TEXT, chcocncr TEXT, chccopd1 TEXT, havarth3 TEXT, addepev2 TEXT, chckidny TEXT, diabete3 TEXT, diabage2 TEXT, lastden3 TEXT, rmvteth3 TEXT, veteran3 TEXT, marital TEXT, children TEXT, educa TEXT, employ1 TEXT, income2 TEXT, weight2 TEXT, height3 TEXT, numhhol2 TEXT, numphon2 TEXT, cpdemo1 TEXT, internet TEXT, renthom1 TEXT, sex TEXT, pregnant TEXT, qlactlm2 TEXT, useequip TEXT, blind TEXT, decide TEXT, diffwalk TEXT, diffdres TEXT, diffalon TEXT, smoke100 TEXT, smokday2 TEXT, stopsmk2 TEXT, las

Now that we have this table for brfss_2014 loaded, let's go ahead and query the database for the information that is interesting.

In [52]:
# Postgres info to connect
connection_args = {
    'host': '54.67.118.82', # my public ip address
    'user': 'ubuntu',    # username
    'dbname': 'baseball',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

connection = pg.connect(**connection_args)

OperationalError: could not connect to server: Operation timed out
	Is the server running on host "54.67.118.82" and accepting
	TCP/IP connections on port 5432?


In [ ]:
diabetes_query = "SELECT xstate, diabete3 FROM brfss_2014"

In [4]:
diabetes_df = pd_sql.read_sql(diabetes_query, connection)

In [5]:
diabetes_df[diabetes_df.xstate == '1']['diabete3'].value_counts()

3    6892
1    1528
4     157
2      62
7       9
9       4
Name: diabete3, dtype: int64

In [59]:
diabetes_df[diabetes_df.xstate == '1']['diabete3'].value_counts(normalize=True)

3    0.796579
1    0.176607
4    0.018146
2    0.007166
7    0.001040
9    0.000462
Name: diabete3, dtype: float64

In [6]:
diabetes_weights_query = "SELECT xstate, diabete3, xllcpwt2, xllcpwt FROM brfss_2014"

In [7]:
diabetes_weights_df = pd_sql.read_sql(diabetes_weights_query, connection)

In [8]:
diabetes_weights_df.head()

,xstate,diabete3,xllcpwt2,xllcpwt
0,1,3,124.3773858077,266.861265396154
1,1,3,121.624464566304,77.5983856870883
2,1,3,175.953264468735,108.943346379677
3,1,3,263.929896703102,259.031555509853
4,1,3,263.929896703102,334.597982387287


In [66]:
(1528)/8652

0.17660656495607951

In [58]:
diabetes_df[diabetes_df.xstate == '1'].shape

(8652, 2)

Now we'll use psycopg2 and sqlalchemy to upload the brfss_2014 dataframe to psql, which will create the schema. Then we'll copy the csv to the table that got created.

In [14]:
# Postgres info to connect
connection_args = {
    'host': input(), # my public ip address
    'user': 'ubuntu',    # username
    'dbname': 'texas_discharge',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

connection = pg.connect(**connection_args)

In [21]:
connection = connect(**params, dbname='ubuntu')

In [16]:
# Connect and create database, disconnect, and reconnect to the right database
connection = connect(**params, dbname='ubuntu')
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
connection.cursor().execute('CREATE DATABASE brfss;')
connection.close()

In [22]:
from sqlalchemy import create_engine
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/brfss'
engine = create_engine(connection_string)
brfss_2014.to_sql('brfss_2014', engine, index=False)

In [23]:
%%bash
ls ../data/brfss

2014_codebook.pdf
BRFSS_COPD_Syntax.pdf
LLCP2017XPT.zip
brfss2010.csv
brfss2011.csv
brfss2012.csv
brfss2013.csv
brfss2014.csv
diabetes_llcpwt_2014.csv
weighting_the-data_webpage_content.pdf


In [14]:
diabetes_llcpwt_2014_df = pd.read_csv('../data/brfss/diabetes_llcpwt_2014.csv', header=None)

In [15]:
diabetes_llcpwt_2014_df.rename(columns={0: 'state', 1:'response', 2:'llcpwt'}, inplace=True)

In [17]:
dm_llcpwt_groupedby_state = diabetes_llcpwt_2014_df.groupby('state')

In [26]:
alabama_dm_llcpwt = dm_llcpwt_groupedby_state.get_group(1)

In [30]:
# Let's calculate the crude prevalance of how many people in the Alabama sample answered 'yes' to the question,
# have you ever been diagnosed with diabetes? 'yes' is encoded as a '1' as per the codebook.

len(alabama_dm_llcpwt[alabama_dm_llcpwt.response ==1])/(len(alabama_dm_llcpwt))

0.17660656495607951

Now, let's calculate the weighted percentage; per the diabetes atlas [here](https://gis.cdc.gov/grasp/diabetes/DiabetesAtlas.html), this should correspond to 11.8% in 2014.


In [37]:
sum(alabama_dm_llcpwt[alabama_dm_llcpwt.response == 1]['llcpwt'])/sum(alabama_dm_llcpwt['llcpwt'])

0.1292790230114006

In [38]:
california_dm_llcpwt = dm_llcpwt_groupedby_state.get_group(6)

In [40]:
len(california_dm_llcpwt[california_dm_llcpwt.response ==1])/(len(california_dm_llcpwt))

0.11096014492753623

In [41]:
sum(california_dm_llcpwt[california_dm_llcpwt.response == 1]['llcpwt'])/sum(california_dm_llcpwt['llcpwt'])

0.10273846760163538

In [42]:
diabetes_all_llcpwt_2014_df = pd.read_csv('../data/brfss/diabetes_all_llcpwt_2014.csv', header=None)

In [45]:
diabetes_all_llcpwt_2014_df.rename(columns={0:'state', 1:'response', 2:'llcpwt', 3:'llcpwt2', 4:'cllcpwt'}, inplace=True)

In [46]:
diabetes_all_llcpwt_2014_df.head()

,state,response,llcpwt,llcpwt2,cllcpwt
0,1,1.0,98.076831,175.953264,NaN
1,1,3.0,137.527171,87.976632,NaN
2,1,3.0,284.683886,175.953264,NaN
3,1,1.0,62.680861,87.976632,NaN
4,1,3.0,186.182279,175.953264,NaN


In [47]:
california = diabetes_all_llcpwt_2014_df[diabetes_all_llcpwt_2014_df.state == 6]

In [48]:
sum(california[california.response == 1].llcpwt2)/sum(california.llcpwt2)

0.10015396765289196

In [49]:
alabama = diabetes_all_llcpwt_2014_df[diabetes_all_llcpwt_2014_df.state == 1]
sum(alabama[alabama.response==1].llcpwt2)/sum(alabama.llcpwt2)

0.14163058246407198

In [50]:
sum(diabetes_all_llcpwt_2014_df[diabetes_all_llcpwt_2014_df.response==1].llcpwt)/sum(diabetes_all_llcpwt_2014_df.llcpwt)

0.1052836645584168

## Extracting relevant predictors from BRFSS 1999-2016

BRFSS has a lot of extraneous predictors; we need to actually extract the relevant predictors that we could use for modeling.

In [54]:
%%bash
ls ../data/brfss/csv/

brfss1999.csv
brfss2000.csv
brfss2001.csv
brfss2002.csv
brfss2003.csv
brfss2004.csv
brfss2005.csv
brfss2006.csv
brfss2007.csv
brfss2008.csv
brfss2009.csv
brfss2010.csv
brfss2011.csv
brfss2012.csv
brfss2013.csv
brfss2014.csv
brfss2015.csv
brfss2016.csv
brfss2017.csv


In [55]:
brfss_1999_df = pd.read_csv("../data/brfss/csv/brfss1999.csv")

/Users/joycelee/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
brfss_1999_df.head()

,Unnamed: 0,x.state,x.geostr,x.denstr,x.psu,x.record,idate,imonth,iday,iyear,...,x.rfdracu,x.rfdrchr,x.rfdrdri,x.raw,x.csa,x.wt1,x.poststr,x.finalwt,exerdis1,exerdis2
0,1,1,1,1,10290,1,1201999,1,20,1999,...,1,1,1,2.0,1.0,2.0,709.955703,1419.911405,NaN,NaN
1,2,1,1,1,10992,1,1131999,1,13,1999,...,1,1,1,2.0,1.0,2.0,919.545794,1839.091587,NaN,NaN
2,3,1,1,1,20586,1,2271999,2,27,1999,...,1,1,1,1.0,1.0,1.0,1024.247333,1024.247333,NaN,NaN
3,4,1,1,1,20667,1,2061999,2,6,1999,...,2,1,1,2.0,1.0,2.0,1190.937133,2381.874267,NaN,NaN
4,5,1,1,1,20813,1,2111999,2,11,1999,...,1,1,1,2.0,1.0,2.0,1190.937133,2381.874267,NaN,NaN


Variables to extract from `brfss_1999_df`:

In [ ]:
to_extract = ['x.state', 'genhlth', 'physhlth', 'menthlth', 'poorhlth', 'hlthplan', 'medicar2', 'nocov12', 'pastplan', 'medcost', 'checkup', 'bphigh', 'highgt1', 'toldhi', 'diabetes', 'lastden2', 'rmvteeth', 'smoke100', 'smokeday', 'lastsmok', 'drinkany', 'drinkdri', 'ctycode', 'hadmam', 'howlong', 'hadpap', 'lastpap', 'flushot', 'hadsigm', 'bikehlmt', 'rsnnocov']

Let's try extracting the variable names from the codebooks; not sure how well this is going to work, but there are different python libraries out there to extract text from pdfs. So, let's try it.

In [3]:
%%bash
ls ../data/brfss/codebooks

1999_codebook.pdf
2000_codebook.pdf
2001_codebook.pdf
2002_codebook.pdf
2003_codebook.pdf
2004_codebook.pdf
2005_codebook.pdf
2006_codebook.pdf
2007_codebook.pdf
2008_codebook.pdf
2009_codebook.pdf
2010_codebook.pdf
2011_codebook.pdf
2012_codebook.pdf
2013_codebook.pdf
2014_codebook.pdf
2015_codebook.pdf
2016_codebook.pdf
2017_codebook.pdf


In [5]:
def extract_pdf_string(pdf_path):
    
    pdfFileObj = open(pdf_path, 'rb')
    
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    num_pages =  pdfReader.numPages
    
    pdf_string = ""
    
    for n in range(num_pages):
        pageObj = pdfReader.getPage(n)
        page_string = pageObj.extractText()
        clean_page_string = re.sub("\s+", " ", page_string)
        clean_page_string = clean_page_string.strip()
        pdf_string += clean_page_string
        
    # closing the pdf file object
    pdfFileObj.close()
    return pdf_string

In [223]:
def find_var(words):
    for word in words:
        if word == 'BRFSS' or word == 'SAS' or len(word) <3:
            pass
        elif word[0] == '_':
            return word
        else:
            ratio = sum([letter.isupper() for letter in list(word)])/len(list(word))
            if ratio > 0.6:
                return word

In [168]:
# def extract_variable_name_and_description(pdf_section):
    
#     # We actually want to start searching only after the phrase 'SAS Variable' appears.
#     if 'SAS Variable' in pdf_section:
#         sub_section = pdf_section.split("SAS Variable")
#         words = sub_section[1].split(" ")
#         var_name = find_var(words)
        
#         if 'Question:' in pdf_section:
#             chunks = pdf_section.split("Question:")
#             description = chunks[1]
#             value_limit = description.find("Value")
#             weighted_limit = description.find("Weighted")
#             if value_limit == -1:
#                 limit = weighted_limit
#             elif weighted_limit == -1:
#                 limit = value_limit
#             else:
#                 limit = min(description.find("Value"), description.find("Weighted"))
#             description = description[0:limit]
#             description = description.strip()
#             if description == '':
#                 description = None
#         else:
#             chunks = pdf_section.split(":")

#             find_description = [i for i, chunk in enumerate(chunks) if 'Description' in chunk]        
#             if find_description:
#                 description = chunks[find_description[0] + 1]
#                 limit = min(description.find("Value"), description.find("Weighted"))
#                 description = description[0:limit]
#                 description = description.strip()
#                 if description == '':
#                     description = None
#             else:
#                 description = None
#     else:
#         var_name = None
#         description = None
    
#     return var_name, description

In [191]:
pdf_string = extract_pdf_string('../data/brfss/codebooks/2002_codebook.pdf')

In [195]:
pdf_variables = pdf_string.split("SAS Variable")

In [196]:
pdf_variables

['Behavioral Risk Factor Surveillance System Codebook Report for 2002State FIPS Code Section: 0.01 Record Identification Type: ID Field Column: 1-2 ',
 ' Name: _STATE Prologue: Description: State FIPS Code Value Value Label Frequency Percentage Weighted Percentage 1 Alabama 3,091 1.25 1.55 2 Alaska 2,694 1.09 0.20 4 Arizona 3,227 1.30 1.84 5 Arkansas 3,897 1.57 0.93 6 California 4,215 1.70 11.73 8 Colorado 4,052 1.63 1.54 9 Connecticut 5,563 2.24 1.20 10 Delaware 4,031 1.63 0.28 11 District of Columbia 2,408 0.97 0.21 12 Florida 6,150 2.48 5.94 13 Georgia 5,065 2.04 2.89 15 Hawaii 6,000 2.42 0.43 16 Idaho 5,037 2.03 0.44 17 Illinois 5,242 2.11 4.28 18 Indiana 5,789 2.33 2.10 19 Iowa 3,662 1.48 1.02 20 Kansas 4,602 1.86 0.92 21 Kentucky 7,067 2.85 1.42 22 Louisiana 5,030 2.03 1.51 23 Maine 2,438 0.98 0.46 24 Maryland 4,401 1.77 1.87 25 Massachusetts 7,429 3.00 2.26 26 Michigan 5,933 2.39 3.43 27 Minnesota 4,488 1.81 1.71 28 Mississippi 4,085 1.65 0.97 29 Missouri 4,730 1.91 1.95 30 Mont

Okay, let's try splitting on 'SAS Variable' instead, and find the var that way.

In [203]:
pdf_string = extract_pdf_string('../data/brfss/codebooks/2017_codebook.pdf')
pdf_variables = pdf_string.split("SAS Variable")

In [227]:
def extract_variable_name_and_description(section):
    words = section.split(" ")
    var_name = find_var(words)
    
    if 'Description:' in section:
        subsections = section.split("Description:")
    elif 'Question:' in section:
        subsections = section.split("Question:")
    else:
        description = None
        return var_name, description
        
    description = subsections[1]

    value_limit = description.find("Value")
    weighted_limit = description.find("Weighted")

    if value_limit == -1:
        limit = weighted_limit
    elif weighted_limit == -1:
        limit = value_limit
    else:
        limit = min(description.find("Value"), description.find("Weighted"))

    description = description[0:limit]
    description = description.strip()

    if description == '':
        description = None
    
    return var_name, description

In [124]:
years = range(1999, 2018)

In [228]:
codebook_dfs_dict = {}

for year in log_progress(years):
    pdf_string = extract_pdf_string(f"../data/brfss/codebooks/{year}_codebook.pdf")
    pdf_sections = pdf_string.split("SAS Variable")

    var_desc_array = []
    for section in pdf_sections:
        row = extract_variable_name_and_description(section)
        var_desc_array.append(row)

    var_desc_df = pd.DataFrame(var_desc_array, columns=['var_name', 'description'])
    
    # We'll drop the rows where the variable is none, and then also get rid of duplicate rows where the var_names
    # are the same.
    var_desc_df = var_desc_df.dropna(subset=['var_name']).drop_duplicates(subset=['var_name'])

    codebook_dfs_dict[year] = var_desc_df

In [229]:
with open("../data/pickles/codebook_dfs_dict.pkl", "wb") as f:
    pickle.dump(codebook_dfs_dict, f)

Now, let's try joining these dataframes together and seeing how it goes.

In [45]:
t = pd.merge(codebook_dfs_dict[1999], codebook_dfs_dict[2000], on='var_name')

In [60]:
df_list = []
for year, df in codebook_dfs_dict.items():
    df_list.append(df)

In [57]:
list_of_dfs = [codebook_dfs_dict[1999], codebook_dfs_dict[2000], codebook_dfs_dict[2001]]

In [90]:
df_final = reduce(lambda left,right: pd.merge(left,right,how='outer', on='var_name'), df_list)

In [232]:
master_df = codebook_dfs_dict[1999]
master_df.rename(columns={'description':1999}, inplace=True)
for year, df in codebook_dfs_dict.items():
    if year == 1999:
        pass
    else:
        master_df = pd.merge(master_df, df, how='outer', on='var_name')
        master_df.rename(columns={'description':year}, inplace=True)

In [303]:
master_df[master_df.var_name=='_LLCPWT']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
1189,_LLCPWT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...,Final weight assigned to each respondent: Land...


In [304]:
master_df[master_df.var_name=='_FINALWT']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
243,_FINALWT,None,None,Final weight assigned to each respondent.,Final weight assigned to each respondent,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,Final weight assigned to each respondent (Post...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
intersection_df = codebook_dfs_dict[1999]
intersection_df.rename(columns={'description':1999}, inplace=True)
for year, df in codebook_dfs_dict.items():
    if year == 1999:
        pass
    else:
        intersection_df = pd.merge(master_df, df, on='var_name')
        intersection_df.rename(columns={'description':year}, inplace=True)

In [322]:
intersection_df.head()

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2017
0,_STATE,None,None,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,...,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code,State FIPS Code
1,_PSU,None,None,None,Annual Sequence Number,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...,Primary Sampling Unit (Equal to Annual Sequenc...
2,IDATE,None,None,None,None,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,...,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date,Interview Date
3,IMONTH,None,None,Month of interview,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,...,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month,Interview Month
4,IDAY,None,None,Calendar date of interview,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,...,Interview Day,Inte rview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day,Interview Day


In [335]:
majority_vars = intersection_df[intersection_df.isna().sum(axis=1) <= 5]

In [330]:
def make_consensus_var_desc_dict(df):
    raw_dict = dict(pd.DataFrame.transpose(df.set_index('var_name')))
    clean_dict = {}
    for key, value in raw_dict.items():
        if value.mode().empty:
            clean_dict[key] = None
        else:
            clean_dict[key] = value.mode()[0]
    return clean_dict

In [300]:
d = make_consensus_var_desc_dict(master_df)

In [336]:
c = make_consensus_var_desc_dict(majority_vars)

In [337]:
len(c)

64

In [338]:
print(c.keys())

dict_keys(['_STATE', '_PSU', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE', 'SEQNO', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'MEDCOST', 'SMOKE100', 'MARITAL', 'EDUCA', 'INCOME2', 'SEX', 'PREGNANT', 'INSULIN', 'BLDSUGAR', 'DOCTDIAB', 'FEETCHK', 'EYEEXAM', 'ASTHNOW', '_STSTR', '_AGEG5YR', '_AGE65YR', 'DIABAGE2', 'FEETCHK2', 'DIABEYE', 'DIABEDU', '_TOTINDA', 'FMONTH', 'PERSDOC2', 'EXERANY2', 'STOPSMK2', 'CHILDREN', 'NUMHHOL2', 'NUMPHON2', 'PAINACT2', 'QLMENTL2', 'QLSTRES2', 'QLHLTH2', 'ASTHMAGE', 'ASATTACK', 'ASERVIST', 'ASDRVIST', 'ASRCHKUP', 'ASACTLIM', 'ASYMPTOM', 'ASNOSLEP', '_STRWT', 'QSTVER', 'QSTLANG', '_RFHLTH', '_CHLDCNT', '_EDUCAG', '_INCOMG', 'WEIGHT2', '_AGE_G'])


In [339]:
c

{'_STATE': 'State FIPS Code',
 '_PSU': 'Primary Sampling Unit (Equal to Annual Sequence Number)',
 'IDATE': 'Interview Date',
 'IMONTH': 'Interview Month',
 'IDAY': 'Interview Day',
 'IYEAR': 'Interview Year',
 'DISPCODE': 'Final Disposition',
 'SEQNO': 'Annual Sequence Number',
 'NUMADULT': 'Number of Adults in Household',
 'NUMMEN': 'Number of Adult Men in Household',
 'NUMWOMEN': 'Number of Adult Women in Household',
 'GENHLTH': 'Would you say that in general your health is:',
 'PHYSHLTH': 'Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?',
 'MENTHLTH': 'Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?',
 'POORHLTH': 'During the past 30 days, for about how many days did poor physical or mental health keep you from doing your usual activities, suc

In [341]:
def remove_no_description(var_desc_dict):
    new_dict = {}
    for key, value in var_desc_dict.items():
        if value:
            new_dict[key] = value
    return new_dict

In [346]:
d['DIABETES']

'5.1. [People may] provide regular care or assistance to [someone] who is elderly or has a long -term illness or di sability. During the past month, did you provide any such care or assistance to a family member or friend who is 60+ years of age? Column: 86'

In [344]:
for key, value in d.items():
    if 'DIABE' in key:
        print(key)
        print(value)

DIABETES
5.1. [People may] provide regular care or assistance to [someone] who is elderly or has a long -term illness or di sability. During the past month, did you provide any such care or assistance to a family member or friend who is 60+ years of age? Column: 86
DIABEYE
Has a doctor ever told you that diabetes has affected your eyes or that you had retinopathy?
DIABEDU
Have you ever taken a course or class in how to manage your diabetes yourself?
DIABETE2
Have you ever been told by a doctor that you have diabetes (If "Yes" and respondent is female, ask "Was this only when you were pregnant?". If Respondent says pre -diabetes or borderline diabetes, use response code 4.)
DIABETE3
(Ever told) you have diabetes (If "Yes" and respondent is female, ask "Was this only when you were pregnant?". If Respondent says pre -diabetes or borderline diabetes, use response code 4.)


In [345]:
master_df[master_df.var_name.str.contains("DIABET") == True]

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
40,DIABETES,6.1. How long has it been since you last visit...,5.1. [People may] provide regular care or assi...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
614,DIABETE2,NaN,NaN,NaN,NaN,NaN,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,Have you ever been told by a doctor that you h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,DIABETE3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...","(Ever told) you have diabetes (If ""Yes"" and re...",(Ever told) you have diabetes (If ´Yes´ and re...,(Ever told) you have diabetes (If ´Yes´ and re...


In [121]:
master_df[master_df.var_name.str.contains("CELLFON")==True]

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
486,CELLFON,NaN,NaN,NaN,NaN,NaN,NaN,Cellular Telephone,NaN,Cellular Telephone,Cellular Telephone,Cellular Telephone,Cellular Telephone,Cellular Telephone,Cellular Telephone,NaN,NaN,NaN,None
540,CELLFON1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cellular Telephone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
761,CELLFON3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Is this a cellular telephone? (Telephone servi...,NaN,NaN,None
808,CELLFON4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
810,CELLFON5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [112]:
master_df[master_df.var_name == 'CELLFON']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
486,CELLFON,NaN,NaN,NaN,NaN,NaN,NaN,Cellular Telephone,NaN,Cellular Telephone,Cellular Telephone,Cellular Telephone,Cellular Telephone,Cellular Telephone,Cellular Telephone,NaN,NaN,NaN,None


In [115]:
master_df[master_df.var_name == 'CELLFON3']

,var_name,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
761,CELLFON3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Is this a cellular telephone? (Telephone servi...,NaN,NaN,None


In [340]:
d

{'FIPS': None,
 '_STATE': 'State FIPS Code',
 '_GEOSTR': 'Geographic Stratum Code',
 '_DENSTR': 'Density Stratum',
 '_PSU': 'Primary Sampling Unit (Equal to Annual Sequence Number)',
 '_RECORD': 'Record number',
 'IDATE': 'Interview Date',
 'IMONTH': 'Interview Month',
 'IDAY': 'Interview Day',
 'IYEAR': 'Interview Year',
 'INTVID': 'Interviewer Identification',
 'DISPCODE': 'Final Disposition',
 'WINDDOWN': 'Interview disposition according to Waksberg Design',
 'SEQNO': 'Annual Sequence Number',
 'NATTMPTS': 'Number of Attempts',
 'NRECSEL': 'Number of Sample Records Selected from Stratum',
 'NRECSTR': 'Number of Telephone Numbers in Stratum from Which Sample Was Selected',
 'BPSELWT': None,
 'WAKSS1S2': None,
 'NUMADULT': 'Number of Adults in Household',
 'NUMMEN': 'Number of Adult Men in Household',
 'NUMWOMEN': 'Number of Adult Women in Household',
 'GENHLTH': 'Would you say that in general your health is:',
 'PHYSHLTH': 'Now thinking about your physical health, which includes phys